In [49]:
import os
from dotenv import load_dotenv
import json
import pandas as pd
import requests
import time

In [50]:
load_dotenv()
beakey = os.environ.get('API_KEY_BEA')

In [51]:
method = "GetData"
data_set = "Regional"
tablename='SAINC1'
geo='STATE'
lc='3'
year='2022'
params = f'TableName={tablename}&GeoFips={geo}&LineCode={lc}&Year={year}'


In [52]:
data = requests.get(
        f'https://apps.bea.gov/api/data?&UserID={beakey}' +
        f'&method={method}&datasetname={data_set}' +
        f'&{params}&ResultFormat=JSON'
    ).json()

In [53]:
df_data = pd.DataFrame(data["BEAAPI"]['Results']['Data'])
df_data = df_data.set_index('GeoName')
df_data = df_data.sort_values('DataValue', ascending=False)
df_data.head()

,Code,GeoFips,TimePeriod,CL_UNIT,UNIT_MULT,DataValue,NoteRef
GeoName,,,,,,,
District of Columbia,SAINC1-3,11000,2022,Dollars,0,96092,2
Massachusetts,SAINC1-3,25000,2022,Dollars,0,84551,2
Connecticut,SAINC1-3,09000,2022,Dollars,0,83340,2
New England,SAINC1-3,91000,2022,Dollars,0,78603,2
New Jersey,SAINC1-3,34000,2022,Dollars,0,77206,2


In [54]:
method = "GetData"
data_set = "Regional"
tablename='CAINC4'
geo='COUNTY'
lc='30'
year='2022'

params = f'TableName={tablename}&GeoFips={geo}&LineCode={lc}&Year={year}'

In [55]:
data_two = requests.get(
        f'https://apps.bea.gov/api/data?&UserID={beakey}' +
        f'&method={method}&datasetname={data_set}' +
        f'&{params}&ResultFormat=JSON'
    ).json()

In [56]:
df_data_two = pd.DataFrame(data_two["BEAAPI"]['Results']['Data'])
df_data_two.head()

,Code,GeoFips,GeoName,TimePeriod,CL_UNIT,UNIT_MULT,DataValue,NoteRef
0,CAINC4-30,01001,"Autauga, AL",2022,Dollars,0,49391,4
1,CAINC4-30,01003,"Baldwin, AL",2022,Dollars,0,56747,4
2,CAINC4-30,01005,"Barbour, AL",2022,Dollars,0,40560,4
3,CAINC4-30,01007,"Bibb, AL",2022,Dollars,0,37513,4
4,CAINC4-30,01009,"Blount, AL",2022,Dollars,0,43744,4


In [57]:
county_df = df_data_two[['GeoName','DataValue','TimePeriod', 'GeoFips']]
county_df.head()

,GeoName,DataValue,TimePeriod,GeoFips
0,"Autauga, AL",49391,2022,01001
1,"Baldwin, AL",56747,2022,01003
2,"Barbour, AL",40560,2022,01005
3,"Bibb, AL",37513,2022,01007
4,"Blount, AL",43744,2022,01009


In [58]:
method = "GetData"
data_set = "Regional"
tablename='CAINC4'
geo='COUNTY'
lc='30'
year='2022'

years = [year for year in range(2010,2023)]
def call_api(year):
    time.sleep(5)
    print(year)
    params = f'TableName={tablename}&GeoFips={geo}&LineCode={lc}&Year={year}'
    result = requests.get(
            f'https://apps.bea.gov/api/data?&UserID={beakey}' +
            f'&method={method}&datasetname={data_set}' +
            f'&{params}&ResultFormat=JSON'
        ).json()
    df = pd.DataFrame(result["BEAAPI"]['Results']['Data'])
    df = df.sort_values('DataValue', ascending=False)
    df = df[['GeoName','DataValue','TimePeriod', 'GeoFips']]
    # LEAVING GeoName as is so county and state can be changed outside here
    df = df.rename(columns={'DataValue':'Income','TimePeriod':'Year'})
    return df


In [59]:
results = [call_api(year) for year in years]


2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022


In [60]:
for result in results:
    print(result.head(2))

         GeoName Income  Year GeoFips
2449   Union, SD  91975  2010   46127
319   Pitkin, CO  87923  2010   08097
              GeoName Income  Year GeoFips
2756  Shackelford, TX  99771  2011   48417
2445        Sully, SD  97347  2011   46119
        GeoName Income  Year GeoFips
232   Marin, CA  94993  2012   06041
2021  Burke, ND  93672  2012   38013
        GeoName Income  Year GeoFips
232   Marin, CA  95473  2013   06041
2021  Burke, ND  95111  2013   38013
               GeoName Income  Year GeoFips
334      Fairfield, CT  99718  2014   09001
249  San Francisco, CA  96047  2014   06075
                      GeoName Income  Year GeoFips
252             San Mateo, CA  99076  2015   06081
88   North Slope Borough, AK*  94467  2015   02185
              GeoName Income  Year GeoFips
1770      Wheeler, NE  95676  2016   31183
1912  Westchester, NY  92351  2016   36119
              GeoName Income  Year GeoFips
582        Blaine, ID  99587  2017   16013
1912  Westchester, NY  98470  2017 

In [61]:
county_merged_df = pd.concat(results, ignore_index=True)
county_merged_df['Income'] = pd.to_numeric(county_merged_df['Income'])
county_merged_df.head()


,GeoName,Income,Year,GeoFips
0,"Union, SD",91975,2010,46127
1,"Pitkin, CO",87923,2010,08097
2,"Potter, SD",85727,2010,46107
3,"Marin, CA",83247,2010,06041
4,"Arlington, VA",79214,2010,51013


In [62]:
pivot_merged_df = county_merged_df.pivot(index=['GeoFips','GeoName'], columns='Year', values='Income')
pivot_merged_df.head()

,Year,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
GeoFips,GeoName,,,,,,,,,,,,,
01001,"Autauga, AL",33686,34435,35161,35364,36347,38130,39022,39922,41076,42550,45151,48914,49391
01003,"Baldwin, AL",36797,38046,38307,38428,39598,41502,43012,44498,45852,47911,51230,55865,56747
01005,"Barbour, AL",27743,27981,27759,29422,29436,31024,31035,31963,33211,34685,37111,40795,40560
01007,"Bibb, AL",25072,25877,26914,27116,27859,28760,28584,28877,30033,32104,34938,37175,37513
01009,"Blount, AL",27914,28365,29727,30033,31309,32469,32532,33781,35275,36561,38133,42852,43744


In [63]:
pct_change_df = pivot_merged_df.pct_change(axis='columns') * 100
pct_change_df = pct_change_df.reset_index()
pct_change_df.to_csv('./files/data/pct_change.csv', index=False)
pct_change_df.head()


Year,GeoFips,GeoName,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,01001,"Autauga, AL",NaN,2.223476,2.108320,0.577344,2.779663,4.905494,2.339365,2.306391,2.890637,3.588470,6.112808,8.334256,0.975181
1,01003,"Baldwin, AL",NaN,3.394298,0.686012,0.315869,3.044655,4.808324,3.638379,3.454850,3.042833,4.490535,6.927428,9.047433,1.578806
2,01005,"Barbour, AL",NaN,0.857874,-0.793396,5.990850,0.047583,5.394755,0.035456,2.990172,3.904515,4.438289,6.994378,9.926976,-0.576051
3,01007,"Bibb, AL",NaN,3.210753,4.007420,0.750539,2.740080,3.234143,-0.611961,1.025049,4.003186,6.895748,8.827560,6.402771,0.909213
4,01009,"Blount, AL",NaN,1.615677,4.801692,1.029367,4.248660,3.705005,0.194031,3.839297,4.422604,3.645641,4.299664,12.375108,2.081583


In [64]:
selected_years_10_22 = pivot_merged_df[['2010','2022']]
change_10_22 = selected_years_10_22.pct_change(axis='columns')
change_10_22.reset_index()
change_10_22.to_csv('./files/data/selected_change_10_20.csv')
change_10_22.head()


,Year,2010,2022
GeoFips,GeoName,,
01001,"Autauga, AL",NaN,0.466217
01003,"Baldwin, AL",NaN,0.542164
01005,"Barbour, AL",NaN,0.461990
01007,"Bibb, AL",NaN,0.496211
01009,"Blount, AL",NaN,0.567099


In [65]:
method = "GetData"
data_set = "Regional"
tablename='SAINC1'
geo='STATE'
lc='3'

years = [year for year in range(2010,2023)]

results_state = [call_api(year) for year in years]


2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022


In [66]:
state_merged_df = pd.concat(results_state, ignore_index=True)
state_merged_df['Income'] = pd.to_numeric(state_merged_df['Income'])
state_merged_df.head()
pivot_state_merged_df = state_merged_df.pivot(index=['GeoFips','GeoName'], columns='Year', values='Income')
pivot_state_merged_df.head()

,Year,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
GeoFips,GeoName,,,,,,,,,,,,,
00000,United States,40557,42649,44237,44401,46287,48060,48971,51004,53309,55547,59151,64427,65473
01000,Alabama,33849,34887,35564,35713,36729,38197,38712,39975,41330,43004,45882,50054,50920
02000,Alaska *,49543,52415,53340,52252,55098,56714,55535,56499,58997,60569,61894,65596,68664
04000,Arizona,33774,35244,36333,36877,38220,39836,41122,43208,45260,47693,52084,56361,58390
05000,Arkansas,32218,34180,36287,36285,38218,39266,40179,41402,43029,43751,47145,51631,52604


In [67]:
state_selected_years_10_22 = pivot_state_merged_df[['2019','2022']]
state_change_10_22 = state_selected_years_10_22.pct_change(axis='columns') * 100
state_change_10_22 = state_change_10_22.reset_index()
state_change_10_22_drop = state_change_10_22.drop(index=0)
state_change_10_22_drop.to_csv('./files/data/state_selected_change_10_20.csv')
state_change_10_22_drop.head()

Year,GeoFips,GeoName,2019,2022
1,01000,Alabama,NaN,18.407590
2,02000,Alaska *,NaN,13.364923
3,04000,Arizona,NaN,22.428868
4,05000,Arkansas,NaN,20.234966
5,06000,California,NaN,20.006545
